In [1]:
import torch
import numpy as np
import cv2
import time
import datetime
import pandas as pd
from PIL import Image
from yolov5.models.experimental import attempt_load

from convert_data import collect_data
from class_mapping import yolo_class_mapping, fire_class_mapping, animals_class_mapping
from detection import category
from perform_object import perform_object
from yolo_to_server import send_server

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



#### Version check

In [2]:
print('opencv version: ', cv2.__version__)

opencv version:  4.8.0


In [4]:
camera_name = 'KFQ StarValley'
device = torch.device('cpu')

# Load trained YOLOv5 model weights
yolo_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/yolov5m.pt'
fire_weights_path = '/Users/bongeungu/Downloads/fire3.pt'
animals_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/animals.pt'

yolo_model = attempt_load(yolo_weights_path, device=device)
fire_model = attempt_load(fire_weights_path, device=device)
animals_model = attempt_load(animals_weights_path, device=device)

# Initialize webcam
cap = cv2.VideoCapture(0)

# Create an empty DataFrame to store results
columns = ['Time', 'Class_Name', 'Confidence', 'X1', 'Y1', 'X2', 'Y2', 'action_detection', 'action_category']
results_df = pd.DataFrame(columns=columns)

while True:
    ret, frame = cap.read()  # Read frame from the webcam
    captured_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # 상황 발생 시간 기록
    
    # Preprocess the frame
    image = Image.fromarray(frame)
    image = image.resize((640, 640))  # Resize the image to the model's input size
    image_tensor = torch.from_numpy(np.array(image)).float().permute(2, 0, 1) / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)

    # Perform object detection
    results = perform_object(image_tensor, 0.65, yolo_model)
    results_df = collect_data(results, frame, yolo_class_mapping, results_df, captured_time)
    #print(results_df)
    
    results = perform_object(image_tensor, 0.65, fire_model)
    results_df = collect_data(results, frame, fire_class_mapping, results_df, captured_time)
    #print(results_df)
    
    results = perform_object(image_tensor, 0.65, animals_model)
    results_df = collect_data(results, frame, animals_class_mapping, results_df, captured_time)
    #print(results_df)
    
    results_df = category(results_df)

    # Display the frame with bounding boxes and labels
    cv2.imshow(camera_name, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit the loop
        break
    
    send_server(results_df, camera_name)
    
    time.sleep(2) # 2초 간격으로 사진 캡쳐(30 frames per 1 miniute)

# Save the DataFrame to a CSV file
results_df.to_csv('object_detection_results.csv', index=False)

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Fusing layers... 


YOLOv5m summary: 369 layers, 21190557 parameters, 0 gradients
Fusing layers... 
YOLOv5m summary: 369 layers, 20875359 parameters, 0 gradients, 48.2 GFLOPs
Fusing layers... 
YOLOv5m summary: 369 layers, 20887482 parameters, 0 gradients, 48.3 GFLOPs
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_detection'][i] = 1
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_category'][i] = 'person detected'


{'cctv_id': 'KFQ StarValley', 'event_item': 'person', 'event_time': '2023-08-17 14:41:38', 'event_type': 'P01', 'event_description': 'person detected'}
Record added successfully.


/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_detection'][i] = 1
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_category'][i] = 'person detected'
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action

{'cctv_id': 'KFQ StarValley', 'event_item': 'person', 'event_time': '2023-08-17 14:41:38', 'event_type': 'P01', 'event_description': 'person detected'}
Record added successfully.
{'cctv_id': 'KFQ StarValley', 'event_item': 'person', 'event_time': '2023-08-17 14:41:41', 'event_type': 'P01', 'event_description': 'person detected'}
Record added successfully.


...

/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_detection'][i] = 1
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_category'][i] = 'person detected'
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action

{'cctv_id': 'KFQ StarValley', 'event_item': 'person', 'event_time': '2023-08-17 14:41:38', 'event_type': 'P01', 'event_description': 'person detected'}
Failed to add record.
502
<!doctype html>
<html>
<head>
    <style type="text/css">
        [class^="ant-"]::-ms-clear,
        [class*="ant-"]::-ms-clear {
            display: none;
        }
        [class^="ant-"],
        [class*="ant-"],
        [class^="ant-"] *,
        [class*="ant-"] *,
        [class^="ant-"] *::before,
        [class*="ant-"] *::before,
        [class^="ant-"] *::after,
        [class*="ant-"] *::after {
            box-sizing: border-box;
        }
        html,
        body {
            width: 100%;
            height: 100%;
        }
        *,
        *::before,
        *::after {
            box-sizing: border-box;
        }
        html {
            font-family: sans-serif;
            line-height: 1.15;
            -webkit-text-size-adjust: 100%;
            -ms-text-size-adjust: 100%;
            -

/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_detection'][i] = 1
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action_category'][i] = 'person detected'
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action

{'cctv_id': 'KFQ StarValley', 'event_item': 'person', 'event_time': '2023-08-17 14:41:38', 'event_type': 'P01', 'event_description': 'person detected'}
Failed to add record.
502
<!doctype html>
<html>
<head>
    <style type="text/css">
        [class^="ant-"]::-ms-clear,
        [class*="ant-"]::-ms-clear {
            display: none;
        }
        [class^="ant-"],
        [class*="ant-"],
        [class^="ant-"] *,
        [class*="ant-"] *,
        [class^="ant-"] *::before,
        [class*="ant-"] *::before,
        [class^="ant-"] *::after,
        [class*="ant-"] *::after {
            box-sizing: border-box;
        }
        html,
        body {
            width: 100%;
            height: 100%;
        }
        *,
        *::before,
        *::after {
            box-sizing: border-box;
        }
        html {
            font-family: sans-serif;
            line-height: 1.15;
            -webkit-text-size-adjust: 100%;
            -ms-text-size-adjust: 100%;
            -

KeyboardInterrupt: 